# Smart Chunking Strategies - Ungraph

Este notebook demuestra el uso de `smart_chunk()` para seleccionar automáticamente la mejor estrategia de chunking.

## Objetivos

1. **Smart chunking automático** - Selección automática de estrategia
2. **Comparación de estrategias** - Fixed-size, lexical, semantic, hierarchical
3. **Análisis de recomendaciones** - Entender por qué se selecciona cada estrategia
4. **Selección manual** - Forzar una estrategia preferida

**Referencias:**
- [Smart Chunking Example](./5.%20Smart%20Chunking%20Example.md)


In [ ]:
def add_src_to_path(path_folder: str):
    import sys
    from pathlib import Path
    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/data")

try:
    import ungraph
except ImportError:
    import src
    ungraph = src

from infrastructure.services.langchain_chunking_service import LangChainChunkingService
from domain.entities.document import Document

print(f"📦 Ungraph version: {ungraph.__version__}")


## Parte 1: Smart Chunking Automático

Usemos `smart_chunk()` para seleccionar automáticamente la mejor estrategia.


In [ ]:
# Crear documento de ejemplo
content = """# Machine Learning Fundamentals

## Introduction
Machine learning is a subset of artificial intelligence that enables systems to learn from data.

## Supervised Learning
Supervised learning uses labeled data to train models. Examples include classification and regression.

## Unsupervised Learning
Unsupervised learning finds patterns in unlabeled data. Clustering is a common technique.
"""

doc = Document.create(content=content, filename="ml_basics.md", file_type="markdown")
chunking_service = LangChainChunkingService()

# Smart chunking automático
chunks, metadata = chunking_service.smart_chunk(doc)

print(f"✅ Estrategia seleccionada: {metadata.get('strategy')}")
print(f"📊 Chunks generados: {metadata.get('num_chunks')}")
print(f"📏 Chunk size: {metadata.get('chunk_size')}")
print(f"🔄 Overlap: {metadata.get('chunk_overlap')}")
print(f"\n📝 Primeros 3 chunks:")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\n{i}. {chunk.page_content[:150]}...")


## Parte 2: Comparación de Estrategias

Comparemos diferentes estrategias de chunking con el mismo documento.


In [ ]:
# Comparar estrategias
strategies = ["fixed-size", "lexical", "semantic", "hierarchical"]

print("🔍 Comparando estrategias de chunking:\n")
for strategy in strategies:
    chunks, metadata = chunking_service.smart_chunk(doc, preferred_strategy=strategy)
    print(f"{strategy.upper()}:")
    print(f"  Chunks: {len(chunks)}")
    print(f"  Chunk size promedio: {sum(len(c.page_content) for c in chunks) / len(chunks):.0f} chars")
    print()


## Resumen

### Estrategias Disponibles

- **fixed-size**: Chunks de tamaño fijo (rápido, simple)
- **lexical**: Respeta límites de palabras/párrafos (mejor para texto estructurado)
- **semantic**: Agrupa contenido semánticamente relacionado (mejor calidad)
- **hierarchical**: Estructura jerárquica (mejor para documentos con secciones)

### Mejores Prácticas

1. Usar `smart_chunk()` para selección automática
2. Comparar estrategias para encontrar la mejor para tu caso
3. Ajustar `chunk_size` y `chunk_overlap` según necesidad
